In [1]:
from datasets import load_dataset
import pygments
import pandas as pd

/root/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The list comes from the web page for the dataset

In [2]:
ALL_LANGS = [
    'ada', 'agda', 'alloy', 'antlr', 'applescript', 'assembly', 'augeas', 'awk', 'batchfile', 'bison', 'bluespec', 'c',
'c++', 'c-sharp', 'clojure', 'cmake', 'coffeescript', 'common-lisp', 'css', 'cuda', 'dart', 'dockerfile', 'elixir',
'elm', 'emacs-lisp','erlang', 'f-sharp', 'fortran', 'glsl', 'go', 'groovy', 'haskell','html', 'idris', 'isabelle', 'java', 
'java-server-pages', 'javascript', 'julia', 'kotlin', 'lean', 'literate-agda', 'literate-coffeescript', 'literate-haskell',
 'lua', 'makefile', 'maple', 'markdown', 'mathematica', 'matlab', 'ocaml', 'pascal', 'perl', 'php', 'powershell', 'prolog',
  'protocol-buffer', 'python', 'r', 'racket', 'restructuredtext', 'rmarkdown', 'ruby', 'rust', 'sas', 'scala', 'scheme', 
  'shell', 'smalltalk', 'solidity', 'sparql', 'sql', 'stan', 'standard-ml', 'stata', 'systemverilog', 'tcl', 'tcsh', 'tex', 
  'thrift', 'typescript', 'verilog', 'vhdl', 'visual-basic', 'xslt', 'yacc', 'zig'
]

In [3]:
ds = {lang: load_dataset("bigcode/the-stack-smol-xs", lang, split="train") for lang in ALL_LANGS}

Found cached dataset the-stack-smol-xs (/root/.cache/huggingface/datasets/bigcode___the-stack-smol-xs/ada/1.0.0/ac1274377605fe676fc7ec99e82e11f22ee29fcc9d3297d78193aca3ac4ad16f)
Found cached dataset the-stack-smol-xs (/root/.cache/huggingface/datasets/bigcode___the-stack-smol-xs/agda/1.0.0/ac1274377605fe676fc7ec99e82e11f22ee29fcc9d3297d78193aca3ac4ad16f)
Found cached dataset the-stack-smol-xs (/root/.cache/huggingface/datasets/bigcode___the-stack-smol-xs/alloy/1.0.0/ac1274377605fe676fc7ec99e82e11f22ee29fcc9d3297d78193aca3ac4ad16f)
Found cached dataset the-stack-smol-xs (/root/.cache/huggingface/datasets/bigcode___the-stack-smol-xs/antlr/1.0.0/ac1274377605fe676fc7ec99e82e11f22ee29fcc9d3297d78193aca3ac4ad16f)
Found cached dataset the-stack-smol-xs (/root/.cache/huggingface/datasets/bigcode___the-stack-smol-xs/applescript/1.0.0/ac1274377605fe676fc7ec99e82e11f22ee29fcc9d3297d78193aca3ac4ad16f)
Found cached dataset the-stack-smol-xs (/root/.cache/huggingface/datasets/bigcode___the-stack-smo

In [5]:
lmap = {'c-sharp':'csharp', 'f-sharp':'fsharp', 'standard-ml':'sml', 'batchfile':'batch','java-server-pages':'jsp'}

def pygments_language_id_to_thestack_language_id(str):
    if str in lmap:
        return lmap[str]
    return str

def can_lex_without_errors(lexer, contents: str):
    tokens = pygments.lex(contents, lexer)
    for (tok_type, tok_text) in tokens:
        if tok_type == pygments.token.Token.Error:
            return False
    return True

In [8]:
ds["ada"][0].keys()

dict_keys(['content', 'lang', 'size', 'ext', 'max_stars_count', 'avg_line_length', 'max_line_length', 'alphanum_fraction'])

In [11]:
from pathlib import Path

for language in ds:
    try:
        lexer = pygments.lexers.get_lexer_by_name(pygments_language_id_to_thestack_language_id(language))
    except pygments.util.ClassNotFound:
        print(f"no lexer for {language}")
        continue
    for (index, sample) in enumerate(ds[language]):
        if not can_lex_without_errors(lexer, sample["content"]):
            dir_path = Path("unlexable_files") / language
            if not dir_path.exists():
                dir_path.mkdir()
            (dir_path / f"{index}.{sample['ext']}").write_text(sample["content"])

no lexer for assembly
no lexer for bison
no lexer for bluespec
no lexer for literate-coffeescript
no lexer for maple
no lexer for protocol-buffer
no lexer for rmarkdown
no lexer for visual-basic
no lexer for yacc


In [54]:
errors_df = pd.DataFrame(errors).drop(columns=["index"])
errors_df.groupby("language").size()


language
ada                  8
agda                 8
alloy                6
antlr               37
applescript         10
awk                 24
c                   11
cmake                5
coffeescript         2
common-lisp          2
css                 30
dart                 3
dockerfile           2
elixir              24
elm                  8
emacs-lisp           6
erlang               3
fortran             26
glsl                33
groovy               1
haskell              4
html                 9
idris               14
javascript          17
julia                3
kotlin              13
literate-haskell     9
lua                  3
makefile             8
markdown             5
mathematica         33
ocaml                1
pascal               5
perl                 9
powershell           1
prolog               9
r                    3
racket              10
sas                 19
scala                3
scheme               7
smalltalk           42
solidity            29
sp